## *Validation of the `champ_morceaux` field*

This notebook tests the new `champ_morceaux` piecewise field in TRUST.
It confirms that the composite field matches each original field within its designated zone.

---

### The test case

1. **Syntax recap**  
   ```text
   champ_morceaux <pb>
   {
       defaut       <field_0>
       <subdomain1> <field_1>
       <subdomain2> <field_2>
       …
   }
   ```  
   Any `<field_i>` may be *any* existing TRUST field type (`champ_uniforme`, `champ_fonc_med`, `champ_input_P0`, *etc.*).

2. **Numerical comparison** – cell‑by‑cell and zone‑by‑zone checks that  
   `champ_morceaux(x) == field_i(x)` wherever `x ∈ subdomain_i`.

In [ ]:
from trustutils import run
run.useMEDCoupling()
import medcoupling as mc

def build_mesh_and_field(nx, mesh_path):
    x = mc.DataArrayDouble([i / nx for i in range(nx + 1)])
    y = mc.DataArrayDouble([0, 1])
    mccm = mc.MEDCouplingCMesh()
    mccm.setCoords(x, y)
    mcum = mccm.buildUnstructured()
    mcum.setName("dom")
    mf, _, _, _, F_Ei = mcum.buildDescendingConnectivity()
    mm = mc.MEDFileUMesh.New()
    mm.setMeshAtLevel(0, mcum)
    mm.setMeshAtLevel(-1, mf)

    g = mc.DataArrayInt.New([i for i in range(mf.getNumberOfCells()) if F_Ei[i + 1] == F_Ei[i] + 1])
    g.setName("bord")
    mm.addGroup(-1, g)

    bc = mcum.computeIsoBarycenterOfNodesPerCell()
    grps = {f"z{i}" : [] for i in range(4)}
    for e, b in enumerate(bc):
        for i in range(4):
            if b[0] < (i + 1) * 0.25:
                grps[f"z{i}"].append(e)
                break

    for z, cells in grps.items():
        g = mc.DataArrayInt.New(cells)
        g.setName(z)
        mm.addGroup(0, g)

    mm.write(mesh_path, 2)

    field = mc.MEDCouplingFieldDouble(mc.ON_CELLS)
    field.setMesh(mcum)
    field.setName("linear_field")
    field.fillFromAnalytic(1,"IVec * x")
    mc.WriteFieldUsingAlreadyWrittenMesh(mesh_path, field)

run.introduction("Yannick Gorsse")
run.TRUST_parameters()
run.initBuildDirectory()
build_mesh_and_field(100, f"{run.BUILD_DIRECTORY}/mesh.med")


In [ ]:
run.addCase(".","jdd.data")
run.runCases()

In [ ]:
from trustutils import plot

fig = plot.Graph("Initial time")
xl = [(i + 0.5) / 100 for i in range(100)]
exact = [1 if x < 0.25 else x if x < 0.5 else 5 if x < 0.75 else 10*x for x in xl]
fig.add(xl, exact, label="Exact")
fig.addSegment(f"{run.BUILD_DIRECTORY}/jdd_LAMBDA.son", label="champ_morceaux", marker="o", time=0)

fig = plot.Graph("Final time")
xl = [(i + 0.5) / 100 for i in range(100)]
exact = [1 if x < 0.25 else x if x < 0.5 else 10 if x < 0.75 else 10*x+3 for x in xl]
fig.add(xl, exact, label="Exact")
fig.addSegment(f"{run.BUILD_DIRECTORY}/jdd_LAMBDA.son", label="champ_morceaux", marker="o")